Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import functools
import random
import math
from dataclasses import dataclass
from tqdm import tqdm


logging.basicConfig(level=logging.DEBUG)

In [43]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [44]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [ ]:
def counter(fn): #counter of the teacher
    """Simple decorator for counting number of calls"""
    
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def calculate_total_distance(tsp, dist_matrix):
    return sum(dist_matrix[tsp[i], tsp[i + 1]] for i in range(len(tsp) - 1)) + dist_matrix[tsp[-1], tsp[0]]


def greedy_initialization(dist_matrix):
    num_cities = dist_matrix.shape[0]
    visited = np.full(num_cities, False)
    tsp = []
    city = 0 
    visited[city] = True
    tsp.append(city)

    while len(tsp) < num_cities:
        closest_city = np.argmin(np.where(visited, np.inf, dist_matrix[city]))
        tsp.append(closest_city)
        visited[closest_city] = True
        city = closest_city

    tsp.append(tsp[0])
    return tsp


def inverse_mutation(tsp):
    if len(tsp) < 4:  #Invert at least 4 cities
        return tsp
    
    start, end = sorted(random.sample(range(1, len(tsp) - 1), 2))  #Avoid edges
    tsp[start:end] = tsp[start:end][::-1]  #Actually do the inversion
    return tsp

# Simulated Annealing
def simulated_annealing(initial_tsp, dist_matrix, initial_temp=1000, cooling_rate=0.995, stopping_temp=1e-8):
    current_solution = initial_tsp[:]
    current_distance = calculate_total_distance(current_solution, dist_matrix)
    best_solution = current_solution[:]
    best_distance = current_distance
    
    temp = initial_temp
    
    while temp > stopping_temp:
        new_solution = inverse_mutation(current_solution[:])
        new_distance = calculate_total_distance(new_solution, dist_matrix)

        cost_difference = new_distance - current_distance

        if cost_difference < 0: #check if new solution is better
            current_solution = new_solution
            current_distance = new_distance

            if current_distance < best_distance:
                best_solution = current_solution[:]
                best_distance = current_distance
        else:
            acceptance_probability = math.exp(-cost_difference / temp) #Accept new solution with simmulated annealing probablility
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_distance = new_distance

    
        temp *= cooling_rate #Simmulated annealing cooling down

    return best_solution, best_distance


def run_fast_algorithm(dist_matrix): #main function
    initial_tsp = greedy_initialization(dist_matrix)
    best_solution, best_distance = simulated_annealing(initial_tsp, dist_matrix)
    best_solution_names = [CITIES.at[i, 'name'] for i in best_solution] #get cities names to understand the itinerary
    
    print("Final output:")
    print(" -> ".join(best_solution_names))
    print("Total cost:", best_distance) 
    print("Number of steps:", calculate_total_distance.calls) 


run_fast_algorithm(DIST_MATRIX)


Final output:
Ancona -> Rimini -> Ravenna -> Forlì -> Bologna -> Ferrara -> Vicenza -> Padua -> Venice -> Trieste -> Bolzano -> Trento -> Verona -> Brescia -> Bergamo -> Monza -> Milan -> Novara -> Turin -> Genoa -> Piacenza -> Parma -> Reggio nell'Emilia -> Modena -> Leghorn -> Prato -> Florence -> Perugia -> Terni -> Sassari -> Cagliari -> Palermo -> Catania -> Syracuse -> Reggio di Calabria -> Messina -> Taranto -> Bari -> Andria -> Foggia -> Salerno -> Naples -> Giugliano in Campania -> Latina -> Rome -> Pescara -> Ancona
Total cost: 4320.24534825164
Number of steps: 5055


## Second Greedy Algorithm

In [ ]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


def greedy_solution(): #like before, greedy initialization
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = [city]

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(city)

    tsp.append(tsp[0])
    return tsp

@counter
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]  #go back to the first city
    assert set(tsp) == set(range(len(CITIES)))  #visit a city once

    total_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        total_cost += DIST_MATRIX[c1, c2]
    return total_cost

@dataclass
class Individual:
    genome: np.ndarray  
    fitness: float = None


def fitness(individual):
    path = list(individual.genome) + [individual.genome[0]]  #going back to the first city to make a cycle
    return -tsp_cost(path)  


def parent_selection(population):
    valid_individuals = [ind for ind in population if ind.fitness is not None]
    candidates = sorted(np.random.choice(valid_individuals, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(p1: Individual, p2: Individual): #parent crossover a 2 point crossover
    size = len(p1.genome)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))
    child_genome = np.full(size, -1)
    child_genome[start:end] = p1.genome[start:end]

    fill_from_p2 = [city for city in p2.genome if city not in child_genome[start:end]]
    fill_indices = [i for i in range(size) if child_genome[i] == -1]

    for i, city in zip(fill_indices, fill_from_p2):
        child_genome[i] = city

    return Individual(genome=child_genome)


def mutation(individual): #inversion mutation
    genome = individual.genome.copy()
    start, end = sorted(np.random.choice(len(genome), 2, replace=False))
    genome[start:end] = genome[start:end][::-1]
    
    if np.random.rand() < 0.5:  #random inversion of the parents genome
        idx1, idx2 = np.random.choice(len(genome), 2, replace=False)
        genome[idx1], genome[idx2] = genome[idx2], genome[idx1]  
    
    return Individual(genome=genome)


POPULATION_SIZE = 10
initial_solution = greedy_solution() #first population
population = [Individual(np.roll(initial_solution[:-1], shift=np.random.randint(len(CITIES)))) for _ in range(POPULATION_SIZE)] #mutate the greedy init to get a first  population
for individual in population:
    individual.fitness = fitness(individual)

#chosen parameters to try to have a converging result in a correct amount of time
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10000

for g in tqdm(range(MAX_GENERATIONS)): #Evolutionary algorithm loop
    offspring = []
    for _ in range(OFFSPRING_SIZE):
        if np.random.random() < 0.5:  #mutation rate, greater than rule of thumb because I did not manage with it
            parent = parent_selection(population)
            child = mutation(parent)
        else:  # Croisement
            parent1 = parent_selection(population)
            parent2 = parent_selection(population)
            child = xover(parent1, parent2)
        child.fitness = fitness(child)
        offspring.append(child)

    population.extend(offspring)
    population.sort(key=lambda ind: ind.fitness, reverse=True)
    population = population[:POPULATION_SIZE]  #Select only the two best parents

    best_parent = population[0]
    second_best_parent = population[1]
    best_cost = -best_parent.fitness
    

best_individual = population[0]
best_path = list(best_individual.genome) + [best_individual.genome[0]]
best_solution_names = [CITIES.at[city, 'name'] for city in best_path]  
best_distance = tsp_cost(best_path)  

print("Best offspring foound:")
print(" -> ".join(best_solution_names))
print("Total costs:", best_distance)

print(f"Number of steps: {tsp_cost.calls}")



100%|██████████| 10000/10000 [00:03<00:00, 2533.82it/s]

Best offspring foound:
Parma -> Reggio nell'Emilia -> Modena -> Bologna -> Ferrara -> Ravenna -> Forlì -> Rimini -> Ancona -> Pescara -> Giugliano in Campania -> Naples -> Salerno -> Foggia -> Andria -> Bari -> Taranto -> Messina -> Reggio di Calabria -> Syracuse -> Catania -> Palermo -> Cagliari -> Sassari -> Latina -> Rome -> Terni -> Perugia -> Florence -> Prato -> Leghorn -> Genoa -> Turin -> Novara -> Milan -> Monza -> Bergamo -> Trento -> Bolzano -> Trieste -> Venice -> Padua -> Vicenza -> Verona -> Brescia -> Piacenza -> Parma
Total costs: 4260.041049398426
Number of steps: 40011


my final bugs correction, ameliorations (like the progress bar of the second algorithm) and correction of syntax have been made using chatgpt